In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from datetime import datetime,timedelta

In [2]:
url = "https://music.bugs.co.kr/chart/track/day/total"

userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"

session = requests.Session()

session.headers.update({
        "Referer": "",
        "User-Agent": userAgent,
})

r = session.get(url)

if r.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)

r.encoding = "utf-8"
soup = BeautifulSoup(r.content,"html.parser")


In [3]:
일간차트 = []                    # 수집한 정보를 저장할 빈 리스트
query = datetime(2023,6,1)      # 추출을 시작할 날짜 입력
now = datetime.now()            # 현재 날짜
nowday = now.strftime('%Y%m%d') # 날짜의 형태로 연산이 가능하도록 설정 
days = int(nowday)              # 날짜를 days라는 변수로 초기화

while query < datetime.strptime(nowday, '%Y%m%d'): # 추출을 시작할 날짜가 현재 날짜가 될 때까지 반복
    url = "https://music.bugs.co.kr/chart/track/day/total?chartdate={}".format(query.strftime('%Y%m%d')) # 설정된 현재 날짜를 url에 입력되도록 설정
    r = session.get(url)         # 접속할 객체 생성
    soup = BeautifulSoup(r.content,"html.parser")
    titles = soup.select("p.title")     # p.title 태그를 추출하여 titles 변수에 초기화
    artists = soup.select("p.artist")   # p.artist 태그를 추출하여 artist 변수에 초기화
    albums = soup.select("a.album")     # a.album 태그를 추출하여 album 변수에 초기화

    for i in range(len(titles)):            # titles의 개수만큼 반복
        title = titles[i].text.strip()      # titles의 text를 title 변수에 초기화
        artist = artists[i].text.strip()    # artists의 text를 artist 변수에 초기화
        album = albums[i+1].text.strip()    # albums의 text를 album 변수에 초기화
        rank = i+1      # 한번 반복할때마다 rank 1씩 증가
        if rank ==1:    # rank가 1이 되었을 때 날짜 1증가
            days-=1     # 첫째 날에서 날짜가 1이 증가되므로 1감소

        chart_list = {"날짜": query.strftime('%Y%m%d'),"곡명": title, "아티스트":artist, "앨범명":album, "순위" : rank} 
        일간차트.append(chart_list) # 데이터를 dictionary 형식으로 일간차트 list에 저장
    query += timedelta(days=1)     
df = DataFrame(일간차트)            # 일간차트 list를 DataFrame으로 생성
df.set_index("순위", inplace=True) 
df.to_excel("벅스일간차트_0601-0629.xlsx")        # 일간차트 list를 excel 파일로 생성
df

,날짜,곡명,아티스트,앨범명
순위,,,,
1,20230601,퀸카 (Queencard),(여자)아이들,I feel
2,20230601,Spicy,aespa,MY WORLD - The 3rd Mini Album
3,20230601,I AM,IVE (아이브),I've IVE
4,20230601,UNFORGIVEN (feat. Nile Rodgers),LE SSERAFIM (르세라핌),UNFORGIVEN
5,20230601,Kitsch,IVE (아이브),I've IVE
...,...,...,...,...
96,20230628,NOT SORRY (Feat. pH-1) (Prod. by Slom),이영지,쇼미더머니 11 Episode 3
97,20230628,10X,SHINee (샤이니),HARD - The 8th Album
98,20230628,딱 10CM만,10CM\n\n\r\n10CM,딱 10CM만
